### Normal and Reduction cell 

In [3]:
import tensorflow as tf
import tensorflow.keras
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D, SeparableConv2D, ZeroPadding2D
import tensorflow.keras.models
from tensorflow.keras import models
import os
from tensorflow.keras.layers import Lambda
import numpy as np
from tensorflow.keras import backend

### Load the dataset

In [23]:

batch_size = 80
num_classes = 10
epochs = 10
data_augmentation = True
num_predictions = 20
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_cifar10_trained_model.h5'

# The data, split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Convert class vectors to binary class matrices.
y_train = tensorflow.keras.utils.to_categorical(y_train, num_classes)
y_test = tensorflow.keras.utils.to_categorical(y_test, num_classes)

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


### Define reduction cell

In [29]:
def reduction_cell(img_input, filters, kernelSize, strides):    

    h = img_input
    # comb 2
    
    comb_2_0 = tensorflow.keras.layers.AveragePooling2D((3,3),
                                                        strides=strides,
                                                        padding="same",
                                                        name = 'r_comb_2_0')(h)

    comb_2_0 = tensorflow.keras.layers.UpSampling2D(size=(2, 2),
                                                    data_format=None,
                                                    interpolation='nearest')(comb_2_0)    
    
    comb_2_1 = tensorflow.keras.layers.SeparableConv2D(filters,
                                                       (3,3),
                                                       strides=strides,
                                                       padding='same', 
                                                       name = 'r_comb_2_1')(h)

    comb_2_1 = tensorflow.keras.layers.Activation('relu')(comb_2_1)
    
    comb_2_1 = tensorflow.keras.layers.UpSampling2D(size=(2, 2),
                                                    data_format=None,
                                                    interpolation='nearest')(comb_2_1)
    
    comb_2_2 =  tensorflow.keras.layers.concatenate([comb_2_0, comb_2_1])
    
    # comb 3
    
    comb_3_0 = tensorflow.keras.layers.MaxPooling2D((3, 3),
                                                    strides=(2, 2),
                                                    padding='same',  name = 'r_comb_3_0')(h)

    comb_3_0 = tensorflow.keras.layers.UpSampling2D(size=(2, 2),
                                                    data_format=None,
                                                    interpolation='nearest')(comb_3_0)
    
    comb_3_1 = tensorflow.keras.layers.MaxPooling2D((3, 3),
                                                    strides=(2, 2),
                                                    padding='same', name = 'r_comb_3_1')(h)
    
    comb_3_1 = tensorflow.keras.layers.UpSampling2D(size=(2, 2),
                                                    data_format=None, 
                                                    interpolation='nearest')(comb_3_1)

    comb_3_2 = tensorflow.keras.layers.concatenate([comb_3_0, comb_3_1])
    
    
    #comb 4
    
    comb_4_0 = tensorflow.keras.layers.MaxPooling2D((3, 3),
                                                    strides=(2, 2),
                                                    padding='same',
                                                    name = 'r_comb_4_0')(h)
    
    comb_4_0 = tensorflow.keras.layers.UpSampling2D(size=(2, 2),
                                                    data_format=None,
                                                    interpolation='nearest')(comb_4_0)

    
    comb_4_1 = tensorflow.keras.layers.SeparableConv2D(filters,
                                                       (7,7),
                                                       strides=strides,
                                                       padding='same',
                                                       name = 'r_comb_4_1')(comb_2_2)

    comb_4_1 = tensorflow.keras.layers.Activation('relu')(comb_4_1)
    
    comb_4_1 = tensorflow.keras.layers.UpSampling2D(size=(2, 2), 
                                                    data_format=None, 
                                                    interpolation='nearest')(comb_4_1)
    

    comb_4_2 = tensorflow.keras.layers.concatenate([comb_4_0, comb_4_1])
    
    
     #comb 5
    
    comb_5_0 = tensorflow.keras.layers.SeparableConv2D(filters,
                                                       (7,7),
                                                       strides=strides,
                                                       padding='same',
                                                       name = 'r_comb_5_0')(h)
    
    comb_5_0 = tensorflow.keras.layers.Activation('relu')(comb_5_0)
    
    comb_5_0 = tensorflow.keras.layers.UpSampling2D(size=(2, 2),
                                                    data_format=None,
                                                    interpolation='nearest')(comb_5_0)
    
  
    comb_5_1 = tensorflow.keras.layers.AveragePooling2D((3,3),
                                                        strides=strides,
                                                        padding="same",
                                                        name = 'r_comb_5_1')(h)
    
    comb_5_1 = tensorflow.keras.layers.UpSampling2D(size=(2, 2),
                                                    data_format=None,
                                                    interpolation='nearest')(comb_5_1)

    comb_5_2 = tensorflow.keras.layers.concatenate([comb_5_0, comb_5_1])
    
    # comb 6
         
    comb_6_0 = tensorflow.keras.layers.SeparableConv2D(filters,
                                                       (3,3),
                                                       strides=strides,
                                                       padding='same', 
                                                       name = 'r_comb_6_0')(comb_3_2)
    
    comb_6_0 = tensorflow.keras.layers.Activation('relu')(comb_6_0)
    
    comb_6_0 = tensorflow.keras.layers.UpSampling2D(size=(2, 2),
                                                    data_format=None,
                                                    interpolation='nearest')(comb_6_0)
    
    
    comb_6_1 = tensorflow.keras.layers.Conv2D(filters,
                                              (1,7),
                                              strides=strides,
                                              padding='same', 
                                              name = 'r_comb_6_1_1')(h)
    
    comb_6_1 = tensorflow.keras.layers.Activation('relu')(comb_6_1)
    
    comb_6_1 = tensorflow.keras.layers.UpSampling2D(size=(2, 2),
                                                    data_format=None, 
                                                    interpolation='nearest')(comb_6_1)
    
    comb_6_1 = tensorflow.keras.layers.Conv2D(filters,
                                              (7,1),
                                              strides=strides,
                                              padding='same', 
                                              name = 'r_comb_6_1_2')(comb_6_1)
    
    comb_6_1 = tensorflow.keras.layers.Activation('relu')(comb_6_1)    
    
    comb_6_1 = tensorflow.keras.layers.UpSampling2D(size=(2, 2), 
                                                    data_format=None, 
                                                    interpolation='nearest')(comb_6_1)
    
    comb_6_2 = tensorflow.keras.layers.concatenate([comb_6_0, comb_6_1])
    
    comb_7_2 = tensorflow.keras.layers.concatenate([comb_4_2, comb_5_2, comb_6_2])    
        
    return comb_7_2, img_input

### Define normal cell

In [30]:
def normal_cell(img_input, filters, kernelSize, strides):
    
    h  = img_input

    # comb 2
    comb_2_0 = tensorflow.keras.layers.AveragePooling2D(kernelSize,
                                                        strides=strides,
                                                        name='n_comb_2_0',
                                                        padding="same")(h)

    comb_2_1 = tensorflow.keras.layers.MaxPooling2D((3, 3),
                                                    strides=(1, 1),
                                                    padding='same',
                                                    name='n_comb_2_1')(h)

    comb_2_2 = tensorflow.keras.layers.concatenate([comb_2_0, comb_2_1])

    # comb 3        
    comb_3_0 = comb_2_0
    
    comb_3_1 = tensorflow.keras.layers.AveragePooling2D((3,3),
                                                        strides=(1,1),
                                                        name='n_comb_3_1',
                                                        padding="same")(h)

    comb_3_2 = tensorflow.keras.layers.concatenate([comb_3_0, comb_3_1])

    # comb 4
    comb_4_1 = tensorflow.keras.layers.SeparableConv2D(filters,
                                                       (3,3),
                                                       padding = 'same',
                                                       name='n_comb_4_1')(comb_3_2)

    comb_4_1 = tensorflow.keras.layers.Activation('relu')(comb_4_1)


    comb_4_0 = tensorflow.keras.layers.SeparableConv2D(filters,
                                                       (5, 5),
                                                       padding = 'same',
                                                       name='n_comb_4_0')(comb_2_2)

    comb_4_0 = tensorflow.keras.layers.Activation('relu')(comb_4_0)

    comb_4_2 =  tensorflow.keras.layers.concatenate([comb_4_0, comb_4_1])

    # comb 5
    comb_5_0 = tensorflow.keras.layers.SeparableConv2D(filters,
                                                       (3,3),
                                                       padding = 'same',
                                                       name='n_comb_5_0')(comb_3_2)

    comb_5_0 = tensorflow.keras.layers.Activation('relu')(comb_5_0)

    comb_5_1 = comb_3_2

    comb_5_2 = tensorflow.keras.layers.concatenate([comb_5_0, comb_5_1])

    # comb 6
    comb_6_0 = tensorflow.keras.layers.AveragePooling2D(kernelSize,
                                                        strides=strides,
                                                        name='n_comb_6_0',
                                                        padding="same")(comb_4_2)

    comb_6_1 = tensorflow.keras.layers.SeparableConv2D(filters,
                                                       (3,3),
                                                       padding = 'same',
                                                       name='n_comb_6_1')(comb_3_2)

    comb_6_1 = tensorflow.keras.layers.Activation('relu')(comb_6_1)

    comb_6_2 = tensorflow.keras.layers.concatenate([comb_6_0, comb_6_1])

    comb_7_out = tensorflow.keras.layers.concatenate([comb_5_2, comb_6_2])
    
    return comb_7_out, img_input

### Define Model(AmoebaNet)

In [ ]:
def amoebaNet(input_tensor):
    
    h = tensorflow.keras.layers.Conv2D(filters, (1, 1),
                                       strides=(2, 2),
                                       padding='same',
                                       input_shape=(32,32,3))(input_tensor)
    
    h = tensorflow.keras.layers.UpSampling2D(size=(1, 1),
                                             data_format=None,
                                             interpolation='nearest')(input_tensor)    
    
    output,input_h = reduction_cell(h,
                                    filters = 32,
                                    kernelSize = (3,3),
                                    strides =(2,2))
    
    output,input_h = normal_cell(output,
                                 filters = 32,
                                 kernelSize = (3,3) ,
                                 strides =(1,1))

    output = tensorflow.keras.layers.Flatten()(output)
    
    softmax_output = tensorflow.keras.layers.Dense(10, activation='softmax')(output)        
    
    model = models.Model(input_tensor, softmax_output, name='ameoba net')        
    
    return model

In [31]:
filters =32
x = x_train[:batch_size]
x=tf.to_float(x, name='ToFloat')
img_input = tensorflow.keras.layers.Input(shape = (32,32,3), batch_size =batch_size)
print(img_input.shape)
model = amoebaNet(img_input)

(80, 32, 32, 3)


### Network Summary

In [32]:
print(model.summary())

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_9 (InputLayer)            (80, 32, 32, 3)      0                                            
__________________________________________________________________________________________________
up_sampling2d_74 (UpSampling2D) (80, 32, 32, 3)      0           input_9[0][0]                    
__________________________________________________________________________________________________
r_comb_2_1 (SeparableConv2D)    (80, 16, 16, 32)     155         up_sampling2d_74[0][0]           
__________________________________________________________________________________________________
r_comb_2_0 (AveragePooling2D)   (80, 16, 16, 3)      0           up_sampling2d_74[0][0]           
__________________________________________________________________________________________________
activation

### Optimizer and loss

In [157]:
opt = tensorflow.keras.optimizers.RMSprop(lr=0.0001, decay=1e-6, )

model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

### Preprocessing the data

In [ ]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
y_train = y_train

### Train model

In [158]:
model.fit(x_train, y_train,
          batch_size=batch_size,
          validation_data=(x_test, y_test),
          epochs=epochs)

Train on 50000 samples, validate on 10000 samples
Epoch 1/25
50000/50000 [==============================] - 217s 4ms/sample - loss: 1.7301 - acc: 0.3809 - val_loss: 1.5573 - val_acc: 0.4357
Epoch 2/25
50000/50000 [==============================] - 219s 4ms/sample - loss: 1.4748 - acc: 0.4753 - val_loss: 1.3916 - val_acc: 0.5204
Epoch 3/25
50000/50000 [==============================] - 218s 4ms/sample - loss: 1.3745 - acc: 0.5161 - val_loss: 1.3489 - val_acc: 0.5223
Epoch 4/25
50000/50000 [==============================] - 219s 4ms/sample - loss: 1.3064 - acc: 0.5429 - val_loss: 1.2888 - val_acc: 0.5404
Epoch 5/25
50000/50000 [==============================] - 219s 4ms/sample - loss: 1.2508 - acc: 0.5636 - val_loss: 1.2420 - val_acc: 0.5603
Epoch 6/25
50000/50000 [==============================] - 219s 4ms/sample - loss: 1.2044 - acc: 0.5805 - val_loss: 1.1931 - val_acc: 0.5787
Epoch 7/25
50000/50000 [==============================] - 219s 4ms/sample - loss: 1.1624 - acc: 0.5968 - val_l

### Save Model

In [159]:
# Save model and weights
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print('Saved trained model at %s ' % model_path)



Saved trained model at /home/vikas/Desktop/NewFolder/saved_models/keras_cifar10_trained_model.h5 


### Evaluate

In [160]:
# Score trained model.
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

10000/10000 [==============================] - 13s 1ms/sample - loss: 0.9649 - acc: 0.6599
Test loss: 0.9649413471221924
Test accuracy: 0.6599
